In [1]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
#from xgboost import XGBClassifier
from catboost import CatBoostClassifier
#import xgboost as xgb
import catboost


In [37]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, average_precision_score, roc_auc_score, precision_recall_curve, roc_curve, auc

from sklearn.model_selection import cross_val_score

In [3]:
# Get training and testing data
all_train = pd.read_csv("all_loc_train.csv")
all_test = pd.read_csv("all_loc_test.csv")

all_train.head()

,#,label,AAAAA,AAAAC,AAAAG,AAAAT,AAACA,AAACC,AAACG,AAACT,...,TTTCG,TTTCT,TTTGA,TTTGC,TTTGG,TTTGT,TTTTA,TTTTC,TTTTG,TTTTT
0,mRNALoc_403#10320|0|training,0,0.003643,0.000792,0.001426,0.002218,0.001584,0.000950,0.000792,0.000950,...,0.000000,0.002693,0.002059,0.001109,0.001109,0.002534,0.001584,0.001901,0.003168,0.003643
1,mRNALoc_4404#20399|0|training,0,0.002914,0.000291,0.000583,0.000291,0.000583,0.000000,0.000000,0.000583,...,0.000583,0.000291,0.000583,0.000291,0.000000,0.001166,0.000291,0.001166,0.000583,0.000291
2,mRNALoc_3392#17196|0|training,0,0.002291,0.001375,0.000458,0.001375,0.000000,0.000917,0.000458,0.001375,...,0.000917,0.001375,0.000917,0.000458,0.000458,0.000917,0.000917,0.002750,0.000458,0.000458
3,mRNALoc_3095#162989|0|training,0,0.000000,0.000990,0.000495,0.000000,0.000990,0.000990,0.000000,0.000495,...,0.000000,0.000495,0.000990,0.000000,0.001484,0.000495,0.000495,0.000000,0.000000,0.000000
4,mRNALoc_1488#117177|0|training,0,0.013061,0.002488,0.004043,0.004354,0.001451,0.001348,0.000415,0.002902,...,0.000311,0.001762,0.001970,0.000933,0.002591,0.001970,0.002695,0.002384,0.002488,0.003732


In [6]:
all_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2040 entries, 0 to 2039
Columns: 1026 entries, # to TTTTT
dtypes: float64(1024), int64(1), object(1)
memory usage: 16.0+ MB


In [4]:
all_test.head()

,#,label,AAAAA,AAAAC,AAAAG,AAAAT,AAACA,AAACC,AAACG,AAACT,...,TTTCG,TTTCT,TTTGA,TTTGC,TTTGG,TTTGT,TTTTA,TTTTC,TTTTG,TTTTT
0,mRNALoc_16986#79230|0|testing,0,0.008836,0.002834,0.002334,0.003168,0.001667,0.003168,0.000000,0.002001,...,0.000834,0.001834,0.001167,0.000500,0.001667,0.001834,0.001667,0.002501,0.002001,0.011504
1,mRNALoc_8139#3082|0|testing,0,0.009503,0.002501,0.003168,0.006002,0.003001,0.000834,0.000167,0.002668,...,0.000167,0.003168,0.001667,0.001334,0.001667,0.003501,0.005835,0.001667,0.003501,0.005669
2,mRNALoc_19034#8816|0|testing,0,0.006005,0.001168,0.002168,0.001168,0.001668,0.001334,0.000167,0.000334,...,0.000167,0.001168,0.000834,0.001501,0.000667,0.001668,0.002502,0.001001,0.001334,0.002002
3,mRNALoc_19863#9737|0|testing,0,0.005673,0.001335,0.002503,0.002837,0.001502,0.001335,0.000167,0.001001,...,0.000000,0.001168,0.002002,0.001001,0.002670,0.001001,0.001168,0.000834,0.001669,0.001502
4,mRNALoc_19031#8816|0|testing,0,0.006008,0.001168,0.002170,0.001168,0.001669,0.001335,0.000167,0.000334,...,0.000167,0.001168,0.000834,0.001502,0.000668,0.001669,0.002503,0.001001,0.001335,0.002003


In [9]:
all_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Columns: 1026 entries, # to TTTTT
dtypes: float64(1024), int64(1), object(1)
memory usage: 4.0+ MB


In [21]:
# Remove gene name/symbol column
noname_train = all_train.drop("#", axis="columns")
noname_test = all_test.drop("#", axis="columns")

noname_train.tail()

,label,AAAAA,AAAAC,AAAAG,AAAAT,AAACA,AAACC,AAACG,AAACT,AAAGA,...,TTTCG,TTTCT,TTTGA,TTTGC,TTTGG,TTTGT,TTTTA,TTTTC,TTTTG,TTTTT
2035,3,0.004505,0.001855,0.004240,0.004240,0.001855,0.000928,0.000265,0.002385,0.003843,...,0.000265,0.002518,0.002120,0.001723,0.001458,0.002253,0.000928,0.002253,0.002518,0.003710
2036,3,0.003446,0.003048,0.003711,0.002518,0.003711,0.001060,0.001193,0.002253,0.004506,...,0.000265,0.001193,0.000928,0.000398,0.000795,0.000663,0.000398,0.000133,0.000928,0.000398
2037,3,0.001989,0.001061,0.002519,0.002254,0.000663,0.001458,0.000928,0.001193,0.002254,...,0.000133,0.001591,0.001458,0.001193,0.001856,0.000928,0.001458,0.000928,0.001591,0.002386
2038,3,0.003182,0.001193,0.002254,0.000398,0.001326,0.001591,0.000265,0.001326,0.001856,...,0.000265,0.000663,0.001856,0.000928,0.000265,0.000398,0.000530,0.000398,0.000663,0.000398
2039,3,0.010356,0.003054,0.005709,0.004647,0.003850,0.001328,0.000398,0.002390,0.004780,...,0.000266,0.003319,0.001593,0.001593,0.002124,0.003585,0.003850,0.002523,0.001726,0.004780


In [39]:
# Create training and testing input (X) and output (y) data

X_train = noname_train.drop("label", axis="columns")
y_train = noname_train[["label"]]
y_train = y_train.values.ravel() #returns 1D, flattened array

X_test = noname_test.drop("label", axis="columns")
y_test = noname_test[["label"]]
y_test = y_test.values.ravel()

print("Training input data shape:\n", X_train.shape)
print("Training output data shape:\n", y_train.shape, "\n")

print("Testing input data shape:\n", X_test.shape)
print("Testing input data shape:\n", y_test.shape)




Training input data shape:
 (2040, 1024)
Training output data shape:
 (2040,) 

Testing input data shape:
 (510, 1024)
Testing input data shape:
 (510,)


In [36]:
#Build pipeline for models

pipeline_lr = Pipeline([('lr_classifier',LogisticRegression())])

pipeline_dt = Pipeline([('dt_classifier',DecisionTreeClassifier())])

pipeline_gbcl = Pipeline([('gbcl_classifier',GradientBoostingClassifier())])

pipeline_rf = Pipeline([('rf_classifier',RandomForestClassifier())])

pipeline_knn = Pipeline([('knn_classifier',KNeighborsClassifier())])

pipeline_bnb = Pipeline([('bnb_classifer',BernoulliNB())])

pipeline_bag = Pipeline([('bag_classifer',BaggingClassifier())])

pipeline_ada = Pipeline([('bnb_classifer',AdaBoostClassifier())])

pipeline_gnb = Pipeline([('gnb_classifer',GaussianNB())])

pipeline_mlp = Pipeline([('mlp_classifer',MLPClassifier())])

pipeline_sgd = Pipeline([('sgd_classifer',SGDClassifier())])

#pipeline_xgb = Pipeline([('xgb_classifer',XGBClassifier())])

pipeline_cat = Pipeline([('cat_classifier',CatBoostClassifier(verbose=False))])

# List of all the pipelines
pipelines = [pipeline_lr, pipeline_dt, pipeline_gbcl, pipeline_rf, pipeline_knn, pipeline_bnb, pipeline_bag, pipeline_ada, pipeline_gnb, pipeline_mlp, pipeline_sgd, pipeline_cat]

# Dictionary of pipelines and classifier types for ease of reference
pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: 'Gradient Boost', 3:'RandomForest', 4: 'KNN', 5: 'BN', 6:'Bagging', 7:'Ada Boost', 8:'GaussianNB', 9:'MLP Classifier', 10:'SGD Classifier', 11:'Cat Boost'}


# Fitting the pipelines
for pipe in pipelines:
    pipe.fit(X_train, y_train)

####
# 7 min 43 sec to run
#####

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/pipeline.py:382: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/

In [40]:
#List all results to determine best model
cv_results_accuracy = []
for i, model in enumerate(pipelines):
    cv_score = cross_val_score(model, X_train,y_train, cv=5)
    cv_results_accuracy.append(cv_score)
    print("%s: %f " % (pipe_dict[i], cv_score.mean()))

## ~30 min to run

Logistic Regression: 0.403922 
Decision Tree: 0.502941 
Gradient Boost: 0.614216 
RandomForest: 0.642647 
KNN: 0.503922 
BN: 0.472549 
Bagging: 0.573529 
Ada Boost: 0.454412 
GaussianNB: 0.499020 


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iteration

MLP Classifier: 0.646569 
SGD Classifier: 0.468627 
Cat Boost: 0.642647 
